# Integer Programming (IP) with PyHUBO

## Problem Overview

Integer Programming (IP) is a fundamental optimization technique where we seek to optimize a linear or nonlinear objective function subject to constraints, with the additional requirement that decision variables must take integer values. In this example, we demonstrate how to model and solve a polynomial Integer Programming problem using PyHUBO's Higher-order Unconstrained Binary Optimization (HUBO) framework.

### Real-World Context

Integer Programming problems arise in numerous applications including:
- **Production Planning**: Determining optimal production quantities (must be whole units)
- **Resource Allocation**: Assigning discrete resources like machines, vehicles, or personnel
- **Network Design**: Selecting optimal network configurations with integer capacity constraints
- **Portfolio Optimization**: Choosing integer numbers of assets or bonds

### Problem Instance

In this example, we'll solve a polynomial IP problem with:
- **2 integer variables** x₀ and x₁, each taking values from {0, 1, 2, 3}
- **Nonlinear objective function**: f(x₀, x₁) = x₀x₁ - 1.5x₀ - 12x₁
- **Polynomial constraint**: x₀x₁ ≤ 3

This tutorial will show you how to:
1. Define integer variable domains using `VariableDictionary`
2. Construct polynomial objective functions with `VariableAssignment`
3. Handle nonlinear constraints through penalty methods
4. Generate HUBO Hamiltonians for integer programming
5. Solve using quantum annealing (OpenJIJ) and classical methods (PyQubo)
6. Validate and interpret solutions

## Import PyHUBO Components

We'll import the three core PyHUBO classes needed for this example:

In [1]:
from pyhubo import VariableAssignment, VariableDictionary, HuboHamiltonian

## Step 1: Define Integer Variable Domains

The first step in PyHUBO is defining the **domain** of our integer programming problem. Unlike binary optimization where variables are restricted to {0, 1}, integer programming allows variables to take discrete integer values from a specified range.

### Problem Variables
- **Variables**: `x_0`, `x_1` (the integer decision variables)
- **Values**: `0`, `1`, `2`, `3` (possible integer values for each variable)

### VariableDictionary for Integer Variables
The `VariableDictionary` automatically:
- Determines the minimum number of qubits needed to represent each integer variable
- Creates a unique binary encoding for each variable-value combination
- Provides mappings between integer values and binary representations

With 4 possible values {0, 1, 2, 3} per variable, we need ⌈log₂(4)⌉ = 2 qubits per integer variable.

**Binary Encoding:**
- 00 → value 0
- 01 → value 1  
- 10 → value 2
- 11 → value 3

In [2]:
# Define the domain: each integer variable can take values {0, 1, 2, 3}
domain = {"x_0": ["0", "1", "2", "3"],
          "x_1": ["0", "1", "2", "3"]}

# Create the variable dictionary
ip_variable_dict = VariableDictionary(domain)

# Let's examine the binary encoding
print("Variable encoding information:")
print(f"Total qubits needed: {ip_variable_dict.get_total_qubits()}")

# Show the binary encoding for each variable-value combination
print("\nBinary encoding:")
for variable in ["x_0", "x_1"]:
    print(f"{variable}:")
    for value in ["0", "1", "2", "3"]:
        # Get the index for this value, then convert to binary representation
        index = ip_variable_dict.get_index(variable, value)
        # Use the internal method to get binary string (with proper bit width)
        binary_string = ip_variable_dict._compute_binary_string(index, ip_variable_dict.nbr_bits[variable])
        print(f"  {variable} = {value} → index: {index}, binary: {binary_string}")

Variable encoding information:
Total qubits needed: 4

Binary encoding:
x_0:
  x_0 = 0 → index: 0, binary: 00
  x_0 = 1 → index: 1, binary: 10
  x_0 = 2 → index: 2, binary: 01
  x_0 = 3 → index: 3, binary: 11
x_1:
  x_1 = 0 → index: 0, binary: 00
  x_1 = 1 → index: 1, binary: 10
  x_1 = 2 → index: 2, binary: 01
  x_1 = 3 → index: 3, binary: 11


## Step 2: Define the Polynomial Objective Function

Our Integer Programming problem has a polynomial objective function that we want to minimize:

$$f(x_0, x_1) = x_0 x_1 - 1.5 x_0 - 12 x_1$$

### Understanding the PyHUBO Approach

In PyHUBO, we need to expand polynomial terms over all possible integer value combinations:

1. **Quadratic term x₀x₁**: For each combination of values that x₀ and x₁ can take, we create an indicator variable
2. **Linear terms**: -1.5x₀ and -12x₁ are expanded similarly over all possible values
3. **VariableAssignment indicators**: `VariableAssignment(x_0, "2")` equals 1 when x₀=2, and 0 otherwise

### Mathematical Expansion

The polynomial x₀x₁ becomes:
- When x₀=0, x₁=0: 0×0×indicator(x₀=0)×indicator(x₁=0)
- When x₀=0, x₁=1: 0×1×indicator(x₀=0)×indicator(x₁=1)
- When x₀=1, x₁=2: 1×2×indicator(x₀=1)×indicator(x₁=2)
- And so on for all 16 combinations...

This natural expansion is what makes PyHUBO powerful for integer programming!

In [3]:
# Build the polynomial objective function: f(x_0, x_1) = x_0 * x_1 - 1.5 * x_0 - 12 * x_1
objective_function = 0

# Term 1: x_0 * x_1 (quadratic term)
print("Building quadratic term x_0 * x_1:")
for val_0 in ["0", "1", "2", "3"]:
    for val_1 in ["0", "1", "2", "3"]:
        # Calculate the contribution: value_0 * value_1
        coefficient = int(val_0) * int(val_1)
        if coefficient != 0:  # Only show non-zero terms
            print(f"  x_0={val_0}, x_1={val_1}: coefficient = {int(val_0)} × {int(val_1)} = {coefficient}")
        # Add term: coefficient × indicator_x0 × indicator_x1
        objective_function += VariableAssignment("x_0", val_0) * VariableAssignment("x_1", val_1) * coefficient

# Term 2: -1.5 * x_0 (linear term)
print("\nBuilding linear term -1.5 * x_0:")
for val_0 in ["0", "1", "2", "3"]:
    coefficient = -1.5 * int(val_0)
    if coefficient != 0:  # Only show non-zero terms
        print(f"  x_0={val_0}: coefficient = -1.5 × {int(val_0)} = {coefficient}")
    # Add term: coefficient × indicator_x0
    objective_function += VariableAssignment("x_0", val_0) * coefficient

# Term 3: -12 * x_1 (linear term)  
print("\nBuilding linear term -12 * x_1:")
for val_1 in ["0", "1", "2", "3"]:
    coefficient = -12 * int(val_1)
    if coefficient != 0:  # Only show non-zero terms
        print(f"  x_1={val_1}: coefficient = -12 × {int(val_1)} = {coefficient}")
    # Add term: coefficient × indicator_x1
    objective_function += VariableAssignment("x_1", val_1) * coefficient

print(f"\nComplete objective function created!")
print("Ready to add constraints...")

Building quadratic term x_0 * x_1:
  x_0=1, x_1=1: coefficient = 1 × 1 = 1
  x_0=1, x_1=2: coefficient = 1 × 2 = 2
  x_0=1, x_1=3: coefficient = 1 × 3 = 3
  x_0=2, x_1=1: coefficient = 2 × 1 = 2
  x_0=2, x_1=2: coefficient = 2 × 2 = 4
  x_0=2, x_1=3: coefficient = 2 × 3 = 6
  x_0=3, x_1=1: coefficient = 3 × 1 = 3
  x_0=3, x_1=2: coefficient = 3 × 2 = 6
  x_0=3, x_1=3: coefficient = 3 × 3 = 9

Building linear term -1.5 * x_0:
  x_0=1: coefficient = -1.5 × 1 = -1.5
  x_0=2: coefficient = -1.5 × 2 = -3.0
  x_0=3: coefficient = -1.5 × 3 = -4.5

Building linear term -12 * x_1:
  x_1=1: coefficient = -12 × 1 = -12
  x_1=2: coefficient = -12 × 2 = -24
  x_1=3: coefficient = -12 × 3 = -36

Complete objective function created!
Ready to add constraints...


## Step 3: Handle Polynomial Constraints

Our Integer Programming problem has a polynomial constraint:

$$x_0 x_1 \leq 3$$

### Constraint Handling in PyHUBO

Since PyHUBO solves **unconstrained** optimization problems, we need to convert constraints into **penalty terms**. The idea is:

1. **Identify infeasible combinations**: Find all (x₀, x₁) pairs where x₀×x₁ > 3
2. **Add penalty terms**: For each infeasible combination, add a large penalty to discourage the solver from selecting it
3. **Lagrange multiplier**: Weight the penalty appropriately so constraint violations are heavily penalized

### Infeasible Combinations

Let's identify which value combinations violate x₀×x₁ ≤ 3:
- (0,0): 0×0 = 0 ≤ 3 ✓
- (0,1): 0×1 = 0 ≤ 3 ✓
- (1,2): 1×2 = 2 ≤ 3 ✓
- (2,2): 2×2 = 4 > 3 ✗ (infeasible)
- (2,3): 2×3 = 6 > 3 ✗ (infeasible)
- (3,2): 3×2 = 6 > 3 ✗ (infeasible)
- (3,3): 3×3 = 9 > 3 ✗ (infeasible)

In [4]:
# Build constraint penalty: penalize combinations where x_0 * x_1 > 3
constraint_penalty = 0

print("Building constraint penalty for x_0 * x_1 ≤ 3:")
print("Infeasible combinations (will be penalized):")

for val_0 in ["0", "1", "2", "3"]:
    for val_1 in ["0", "1", "2", "3"]:
        product = int(val_0) * int(val_1)
        if product > 3:  # Constraint violation
            print(f"  x_0={val_0}, x_1={val_1}: {int(val_0)} × {int(val_1)} = {product} > 3 ✗")
            # Add penalty term for this infeasible assignment
            constraint_penalty += VariableAssignment("x_0", val_0) * VariableAssignment("x_1", val_1)
        elif product <= 3:
            if product > 0:  # Only show non-trivial feasible combinations
                print(f"  x_0={val_0}, x_1={val_1}: {int(val_0)} × {int(val_1)} = {product} ≤ 3 ✓")

# Combine objective with constraint penalty using Lagrange multiplier
lagrange_multiplier = 50  # Weight for constraint enforcement
total_cost_function = objective_function + lagrange_multiplier * constraint_penalty

print(f"\nConstraint penalty terms added!")
print(f"Lagrange multiplier (penalty weight): {lagrange_multiplier}")
print(f"Total cost function = objective + {lagrange_multiplier} × constraint_penalty")

Building constraint penalty for x_0 * x_1 ≤ 3:
Infeasible combinations (will be penalized):
  x_0=1, x_1=1: 1 × 1 = 1 ≤ 3 ✓
  x_0=1, x_1=2: 1 × 2 = 2 ≤ 3 ✓
  x_0=1, x_1=3: 1 × 3 = 3 ≤ 3 ✓
  x_0=2, x_1=1: 2 × 1 = 2 ≤ 3 ✓
  x_0=2, x_1=2: 2 × 2 = 4 > 3 ✗
  x_0=2, x_1=3: 2 × 3 = 6 > 3 ✗
  x_0=3, x_1=1: 3 × 1 = 3 ≤ 3 ✓
  x_0=3, x_1=2: 3 × 2 = 6 > 3 ✗
  x_0=3, x_1=3: 3 × 3 = 9 > 3 ✗

Constraint penalty terms added!
Lagrange multiplier (penalty weight): 50
Total cost function = objective + 50 × constraint_penalty


## Step 4: Generate the HUBO Hamiltonian

Now we combine the total cost function (objective + constraints) with the variable dictionary to create a HUBO Hamiltonian. The `HuboHamiltonian` class automatically:

1. **Expands** `VariableAssignment` objects into binary variable products
2. **Collects** like terms to create polynomial coefficients  
3. **Maps** logical variables to physical qubits using the variable dictionary
4. **Generates** the final HUBO representation ready for quantum or classical solvers

The resulting Hamiltonian will be in the form:
$$H = \sum_i h_i Z_i + \sum_{i<j} J_{ij} Z_i Z_j + \sum_{i<j<k} K_{ijk} Z_i Z_j Z_k + \ldots$$

where $Z_i$ are Pauli-Z operators (or binary variables in classical context).

### Qubit Mapping for Integer Variables
- Qubits 0,1: encode x₀'s value (00→0, 01→1, 10→2, 11→3)
- Qubits 2,3: encode x₁'s value (00→0, 01→1, 10→2, 11→3)

In [5]:
# Create the HUBO Hamiltonian
ip_hubo_hamiltonian = HuboHamiltonian(total_cost_function, ip_variable_dict)

print("HUBO Hamiltonian created successfully!")
print(f"Total qubits: {ip_variable_dict.get_total_qubits()}")

# Show variable-to-qubit mapping
print("\nVariable to qubit mapping:")
qubit_offset = 0
for variable in ["x_0", "x_1"]:
    print(f"{variable} uses {ip_variable_dict.nbr_bits[variable]} qubits:")
    for value in ["0", "1", "2", "3"]:
        index = ip_variable_dict.get_index(variable, value)
        binary_string = ip_variable_dict._compute_binary_string(index, ip_variable_dict.nbr_bits[variable])
        # Show which physical qubits are used for this assignment
        active_qubits = [qubit_offset + i for i, bit in enumerate(binary_string) if bit == '1']
        print(f"  {variable} = {value} → binary: {binary_string}, active qubits: {active_qubits}")
    qubit_offset += ip_variable_dict.nbr_bits[variable]

print("Ready to generate coefficients and solve the optimization problem.")

HUBO Hamiltonian created successfully!
Total qubits: 4

Variable to qubit mapping:
x_0 uses 2 qubits:
  x_0 = 0 → binary: 00, active qubits: []
  x_0 = 1 → binary: 10, active qubits: [0]
  x_0 = 2 → binary: 01, active qubits: [1]
  x_0 = 3 → binary: 11, active qubits: [0, 1]
x_1 uses 2 qubits:
  x_1 = 0 → binary: 00, active qubits: []
  x_1 = 1 → binary: 10, active qubits: [2]
  x_1 = 2 → binary: 01, active qubits: [3]
  x_1 = 3 → binary: 11, active qubits: [2, 3]
Ready to generate coefficients and solve the optimization problem.


## Step 5: Extract HUBO Coefficients

The `get_hamiltonian()` method returns a dictionary containing all the polynomial coefficients. Each key represents a product of qubits, and each value is the corresponding coefficient.

For our integer programming problem, we expect:
- **Constant terms**: From value products like 0×1, 0×2, etc.
- **Linear terms**: Single qubit coefficients from the linear objective terms
- **Quadratic terms**: Two-qubit products from interactions
- **Higher-order terms**: Multi-qubit products from the nonlinear constraint penalties

In [6]:
# Get the HUBO coefficients dictionary
hamiltonian_dict = ip_hubo_hamiltonian.get_hamiltonian()

print("HUBO Hamiltonian coefficients:")
print("=" * 50)

# Organize and display coefficients by order
constant_terms = []
linear_terms = []
quadratic_terms = []
higher_order_terms = []

for terms, coefficient in hamiltonian_dict.items():
    terms = [i for i in terms]
    if len(terms) == 0:
        constant_terms.append((terms, coefficient))
    elif len(terms) == 1:
        linear_terms.append((terms, coefficient))
    elif len(terms) == 2:
        quadratic_terms.append((terms, coefficient))
    else:
        higher_order_terms.append((terms, coefficient))

# Display constant terms
if constant_terms:
    print("\nConstant terms:")
    for terms, coeff in constant_terms:
        print(f"  Constant: {coeff}")

# Display linear terms
if linear_terms:
    print("\nLinear terms:")
    for terms, coeff in linear_terms:
        print(f"  Z_{terms[0]}: {coeff}")

# Display quadratic terms  
if quadratic_terms:
    print("\nQuadratic terms:")
    for terms, coeff in quadratic_terms:
        print(f"  Z_{terms[0]} × Z_{terms[1]}: {coeff}")

# Display higher-order terms
if higher_order_terms:
    print("\nHigher-order terms:")
    for terms, coeff in higher_order_terms[:10]:  # Show first 10 to avoid clutter
        term_str = " × ".join([f"Z_{t}" for t in terms])
        print(f"  {term_str}: {coeff}")
    if len(higher_order_terms) > 10:
        print(f"  ... and {len(higher_order_terms) - 10} more higher-order terms")

print(f"\nTotal number of terms: {len(hamiltonian_dict)}")
print(f"Breakdown: {len(constant_terms)} constant, {len(linear_terms)} linear, {len(quadratic_terms)} quadratic, {len(higher_order_terms)} higher-order")

HUBO Hamiltonian coefficients:

Constant terms:
  Constant: -5.5

Linear terms:
  Z_('x_0', 1): -12.5
  Z_('x_1', 0): 5.25
  Z_('x_1', 1): -2.0

Quadratic terms:
  Z_('x_1', 0) × Z_('x_0', 0): 0.25
  Z_('x_0', 1) × Z_('x_1', 0): 0.5
  Z_('x_1', 1) × Z_('x_0', 0): 0.5
  Z_('x_0', 1) × Z_('x_1', 1): 13.5

Total number of terms: 8
Breakdown: 1 constant, 3 linear, 4 quadratic, 0 higher-order


## Step 6: Validate Solutions

Before solving with quantum/classical annealers, let's verify that our Hamiltonian correctly evaluates known solutions. We'll test a few candidate solutions and check both objective values and constraint satisfaction.

### Finding the Optimal Solution by Analysis

Let's analyze our problem:
- **Objective**: f(x₀, x₁) = x₀x₁ - 1.5x₀ - 12x₁
- **Constraint**: x₀x₁ ≤ 3
- **Domain**: x₀, x₁ ∈ {0, 1, 2, 3}

Since we have a small discrete domain, we can evaluate all feasible combinations!

In [7]:
# Evaluate all feasible solutions manually
print("Evaluating all feasible solutions:")
print("x₀  x₁  | x₀×x₁ ≤ 3? | Objective f(x₀,x₁) = x₀×x₁ - 1.5×x₀ - 12×x₁")
print("-" * 65)

best_objective = float('inf')
best_solution = None
feasible_solutions = []

for x0 in [0, 1, 2, 3]:
    for x1 in [0, 1, 2, 3]:
        # Check constraint: x₀×x₁ ≤ 3
        constraint_satisfied = (x0 * x1) <= 3
        
        if constraint_satisfied:
            # Calculate objective value
            objective_value = x0 * x1 - 1.5 * x0 - 12 * x1
            feasible_solutions.append(((x0, x1), objective_value))
            
            # Track best solution
            if objective_value < best_objective:
                best_objective = objective_value
                best_solution = (x0, x1)
            
            print(f" {x0}   {x1}  |    ✓     | {x0}×{x1} - 1.5×{x0} - 12×{x1} = {objective_value}")
        else:
            print(f" {x0}   {x1}  |    ✗     | (infeasible: {x0}×{x1} = {x0*x1} > 3)")

print(f"\nOptimal solution: x₀ = {best_solution[0]}, x₁ = {best_solution[1]}")
print(f"Optimal objective value: {best_objective}")

# Verify this solution using PyHUBO
optimal_solution_dict = {"x_0": str(best_solution[0]), "x_1": str(best_solution[1])}
hubo_cost = ip_hubo_hamiltonian.cost_solution(optimal_solution_dict)
print(f"PyHUBO Hamiltonian evaluation: {hubo_cost}")
print(f"Manual calculation matches PyHUBO: {abs(hubo_cost - best_objective) < 1e-10}")

Evaluating all feasible solutions:
x₀  x₁  | x₀×x₁ ≤ 3? | Objective f(x₀,x₁) = x₀×x₁ - 1.5×x₀ - 12×x₁
-----------------------------------------------------------------
 0   0  |    ✓     | 0×0 - 1.5×0 - 12×0 = 0.0
 0   1  |    ✓     | 0×1 - 1.5×0 - 12×1 = -12.0
 0   2  |    ✓     | 0×2 - 1.5×0 - 12×2 = -24.0
 0   3  |    ✓     | 0×3 - 1.5×0 - 12×3 = -36.0
 1   0  |    ✓     | 1×0 - 1.5×1 - 12×0 = -1.5
 1   1  |    ✓     | 1×1 - 1.5×1 - 12×1 = -12.5
 1   2  |    ✓     | 1×2 - 1.5×1 - 12×2 = -23.5
 1   3  |    ✓     | 1×3 - 1.5×1 - 12×3 = -34.5
 2   0  |    ✓     | 2×0 - 1.5×2 - 12×0 = -3.0
 2   1  |    ✓     | 2×1 - 1.5×2 - 12×1 = -13.0
 2   2  |    ✗     | (infeasible: 2×2 = 4 > 3)
 2   3  |    ✗     | (infeasible: 2×3 = 6 > 3)
 3   0  |    ✓     | 3×0 - 1.5×3 - 12×0 = -4.5
 3   1  |    ✓     | 3×1 - 1.5×3 - 12×1 = -13.5
 3   2  |    ✗     | (infeasible: 3×2 = 6 > 3)
 3   3  |    ✗     | (infeasible: 3×3 = 9 > 3)

Optimal solution: x₀ = 0, x₁ = 3
Optimal objective value: -36.0
PyHUBO H

## Step 7: Solving with OpenJIJ and PyQubo

This section demonstrates how PyHUBO integrates with popular optimization libraries for solving integer programming problems:

1. **OpenJIJ**: Direct HUBO solving using simulated annealing
2. **PyQubo + Neal**: Convert to QUBO format and solve with D-Wave's classical solver

### Why Two Approaches?

- **HUBO (Higher-order)**: Natural representation for polynomial integer programming, no auxiliary variables needed
- **QUBO (Quadratic)**: Compatible with D-Wave quantum annealers and many classical solvers, but requires one-hot constraints

### Expected Results

Based on our manual analysis, we expect:
- **Optimal solution**: x₀ = 0, x₁ = 3  
- **Optimal objective value**: -36
- **All feasible solutions**: 13 different combinations satisfying x₀×x₁ ≤ 3

### Method 1: Direct HUBO Solving with OpenJIJ

In [8]:
# Helper function to convert qubit indices to string format for OpenJIJ
def map_coeff_to_str(coeffs):
    """Convert coefficient tuples to string format required by OpenJIJ"""
    return tuple([coeff[0] + "_" + str(coeff[1]) for coeff in coeffs])

# Export the Hamiltonian in OpenJIJ format
print("Converting HUBO Hamiltonian for OpenJIJ...")
hubo_hamiltonian_dict = ip_hubo_hamiltonian.export_dict(
    map_coeff_to_str, 
    non_standard_definition_pauli_z=True  # Use {-1, +1} spin variables
)

print(f"Exported {len(hubo_hamiltonian_dict)} HUBO terms")

# Solve using OpenJIJ's Simulated Annealing
import openjij as oj

print("Solving with OpenJIJ Simulated Annealing...")
sampler = oj.SASampler()
response = sampler.sample_hubo(
    hubo_hamiltonian_dict, 
    vartype="SPIN",      # Use spin variables {-1, +1}
    num_reads=100,       # Number of optimization runs
    num_sweeps=1000      # Annealing steps per run
).change_vartype("BINARY")  # Convert to binary {0, 1} for interpretation

print("OpenJIJ Results:")
print("=" * 50)

# Show the best few solutions
for i, (sample, energy) in enumerate(zip(response.samples(), response.data_vectors['energy'])):
    if i < 5:  # Show top 5 solutions
        print(f"Solution {i+1}: Energy = {energy}")
        print(f"  Binary assignment: {dict(sample)}")
    else:
        break

# Extract the best solution
best_sample = response.first.sample
best_energy = response.first.energy
print(f"\nBest energy found: {best_energy}")
print(f"Best binary assignment: {dict(best_sample)}")

Converting HUBO Hamiltonian for OpenJIJ...
Exported 8 HUBO terms
Solving with OpenJIJ Simulated Annealing...
OpenJIJ Results:
Solution 1: Energy = -36.0
  Binary assignment: {'x_0_0': np.int8(0), 'x_0_1': np.int8(0), 'x_1_0': np.int8(1), 'x_1_1': np.int8(1)}
Solution 2: Energy = -36.0
  Binary assignment: {'x_0_0': np.int8(0), 'x_0_1': np.int8(0), 'x_1_0': np.int8(1), 'x_1_1': np.int8(1)}
Solution 3: Energy = -36.0
  Binary assignment: {'x_0_0': np.int8(0), 'x_0_1': np.int8(0), 'x_1_0': np.int8(1), 'x_1_1': np.int8(1)}
Solution 4: Energy = -36.0
  Binary assignment: {'x_0_0': np.int8(0), 'x_0_1': np.int8(0), 'x_1_0': np.int8(1), 'x_1_1': np.int8(1)}
Solution 5: Energy = -36.0
  Binary assignment: {'x_0_0': np.int8(0), 'x_0_1': np.int8(0), 'x_1_0': np.int8(1), 'x_1_1': np.int8(1)}

Best energy found: -36.0
Best binary assignment: {'x_0_0': np.int8(0), 'x_0_1': np.int8(0), 'x_1_0': np.int8(1), 'x_1_1': np.int8(1)}


### Interpreting OpenJIJ Results

OpenJIJ returns binary assignments for each qubit. We need to convert these back to meaningful integer variable assignments using the `VariableDictionary`:

In [9]:
# Convert OpenJIJ binary solution back to integer variable assignments
print("Converting binary solution to integer variable assignments:")

# Build solution dictionary from binary assignments
binary_solution = {}
for var_name, bit_value in best_sample.items():
    # Parse the qubit name (format: "x_0_1" means variable x_0, bit 1)
    parts = var_name.split('_')
    if len(parts) == 3:  # variable_index_bit
        variable = f"{parts[0]}_{parts[1]}"  # e.g., "x_0"
        bit = int(parts[2])
        
        if variable not in binary_solution:
            binary_solution[variable] = ['0', '0']  # Initialize with 2 bits
        
        binary_solution[variable][bit] = str(bit_value)

# Convert binary strings to integer values
print("Binary to integer conversion:")
interpreted_solution = {}
for variable in ["x_0", "x_1"]:
    if variable in binary_solution:
        binary_string = ''.join(binary_solution[variable])
        # Convert binary to decimal: e.g., "11" -> 3
        integer_value = int(binary_string, 2)
        interpreted_solution[variable] = str(integer_value)
        print(f"  {variable}: binary {binary_string} → integer {integer_value}")

print("\nOpenJIJ Solution:")
print("=" * 30)
for variable, value in interpreted_solution.items():
    print(f"{variable} = {value}")

# Verify this solution
print(f"\nSolution verification:")
x0_val = int(interpreted_solution.get("x_0", "0"))
x1_val = int(interpreted_solution.get("x_1", "0"))

# Check constraint
constraint_satisfied = (x0_val * x1_val) <= 3
print(f"Constraint x₀×x₁ ≤ 3: {x0_val}×{x1_val} = {x0_val * x1_val} ≤ 3? {constraint_satisfied}")

# Calculate objective
manual_objective = x0_val * x1_val - 1.5 * x0_val - 12 * x1_val
print(f"Objective value: {x0_val}×{x1_val} - 1.5×{x0_val} - 12×{x1_val} = {manual_objective}")

# Verify with PyHUBO
if interpreted_solution:
    hubo_cost = ip_hubo_hamiltonian.cost_solution(interpreted_solution)
    print(f"PyHUBO verification: {hubo_cost}")
    print(f"Manual vs PyHUBO match: {abs(hubo_cost - manual_objective) < 1e-10}")

Converting binary solution to integer variable assignments:
Binary to integer conversion:
  x_0: binary 00 → integer 0
  x_1: binary 11 → integer 3

OpenJIJ Solution:
x_0 = 0
x_1 = 3

Solution verification:
Constraint x₀×x₁ ≤ 3: 0×3 = 0 ≤ 3? True
Objective value: 0×3 - 1.5×0 - 12×3 = -36.0
PyHUBO verification: -36.0
Manual vs PyHUBO match: True


### Method 2: QUBO Conversion with PyQubo + Neal

Now let's demonstrate how PyHUBO can convert integer programming problems to QUBO format for compatibility with D-Wave systems and other QUBO solvers. This approach requires adding one-hot constraints to ensure each integer variable takes exactly one value.

In [10]:
# Convert PyHUBO cost function to PyQubo format
print("Converting to PyQubo format...")
pyqubo_cost_function = total_cost_function.to_pyqubo()
print(f"PyQubo cost function: {type(pyqubo_cost_function)}")

# IMPORTANT: QUBO requires one-hot constraints for integer variables
# Each integer variable must be assigned to exactly one value
print("\nAdding one-hot constraints for integer variables...")
pyqubo_one_hot_penalty = ip_variable_dict.one_hot_penalty()
print(f"One-hot penalty generated successfully")

# Combine objective with one-hot constraints
one_hot_lagrange_penalty = 100  # Weight for one-hot enforcement
print(f"One-hot penalty weight: {one_hot_lagrange_penalty}")

total_model = pyqubo_cost_function + one_hot_lagrange_penalty * pyqubo_one_hot_penalty
print("Compiling PyQubo model...")
model = total_model.compile()
print("✅ PyQubo model compiled successfully!")

print(f"\nModel statistics:")
print(f"  Number of variables: {len(model.variables)}")
print(f"  Variables: {sorted(model.variables)}")

Converting to PyQubo format...
PyQubo cost function: <class 'cpp_pyqubo.Add'>

Adding one-hot constraints for integer variables...
One-hot penalty generated successfully
One-hot penalty weight: 100
Compiling PyQubo model...
✅ PyQubo model compiled successfully!

Model statistics:
  Number of variables: 8
  Variables: ['x_0_0', 'x_0_1', 'x_0_2', 'x_0_3', 'x_1_0', 'x_1_1', 'x_1_2', 'x_1_3']


### Solving the QUBO Model

We'll use D-Wave's Neal simulated annealing sampler to solve the QUBO formulation:

In [11]:
# Solve using D-Wave Neal simulated annealing
import neal

print("Solving QUBO with Neal Simulated Annealing...")
sampler = neal.SimulatedAnnealingSampler()

# Convert to Binary Quadratic Model (BQM) format
bqm = model.to_bqm()
print(f"BQM created with {len(bqm.variables)} variables")

# Run the sampler
sampleset = sampler.sample(bqm, num_reads=100, num_sweeps=1000)
print(f"Sampling completed. {len(sampleset)} samples generated.")

# Decode the results back to logical variables
decoded_samples = model.decode_sampleset(sampleset)
print(f"Decoded {len(decoded_samples)} samples")

# Find the best (lowest energy) sample
best_sample = min(decoded_samples, key=lambda x: x.energy)

print("\nNeal/PyQubo Results:")
print("=" * 30)
print(f"Best energy: {best_sample.energy}")

# Check if the solution satisfies one-hot constraints
print(f"\nConstraint validation:")
for constraint, value in best_sample.constraints().items():
    if hasattr(value, 'keys') and 'penalty' in value:
        print(f"  {constraint}: penalty = {value['penalty']}")
    else:
        print(f"  {constraint}: {value}")

# Display the final integer variable assignments
print(f"\nFinal integer variable assignments:")
solution_dict = {}
for var, val in best_sample.sample.items():
    if val == 1:  # Only show activated variables (one-hot encoding)
        # Parse variable name to extract variable and value
        if '_' in var:
            parts = var.split('_')
            if len(parts) == 2:
                variable, value = parts
                solution_dict[variable] = value
                print(f"  {variable} = {value}")

# Verify the solution cost
if solution_dict:
    verification_cost = ip_hubo_hamiltonian.cost_solution(solution_dict)
    print(f"\nSolution cost verification: {verification_cost}")
    
    # Manual verification
    if "x" in solution_dict and "0" in solution_dict:
        x0_val = int(solution_dict.get("x", "0"))
        x1_val = int(solution_dict.get("0", "0"))
        manual_cost = x0_val * x1_val - 1.5 * x0_val - 12 * x1_val
        print(f"Manual calculation: {manual_cost}")
        
        # Check constraint
        constraint_ok = (x0_val * x1_val) <= 3
        print(f"Constraint satisfied: {constraint_ok}")
else:
    print("No valid solution found - check one-hot constraints")

Solving QUBO with Neal Simulated Annealing...
BQM created with 8 variables
Sampling completed. 100 samples generated.
Decoded 100 samples

Neal/PyQubo Results:
Best energy: -36.0

Constraint validation:

Final integer variable assignments:
No valid solution found - check one-hot constraints


## Summary and Conclusions

🎯 **This tutorial demonstrated PyHUBO's powerful capabilities for Integer Programming:**

### Problem Solved
- **Objective**: Polynomial function f(x₀, x₁) = x₀x₁ - 1.5x₀ - 12x₁
- **Constraint**: Nonlinear constraint x₀x₁ ≤ 3  
- **Domain**: Integer variables x₀, x₁ ∈ {0, 1, 2, 3}
- **Optimal Solution**: x₀ = 0, x₁ = 3 with objective value = -36

### Key PyHUBO Features for Integer Programming

1. **Natural Integer Variable Modeling**: `VariableDictionary` automatically handles integer domain encoding
2. **Polynomial Objective Functions**: Higher-order terms like x₀x₁ are naturally represented without auxiliary variables
3. **Constraint Handling**: Nonlinear constraints converted to penalty terms via Lagrange multipliers
4. **Multiple Solver Integration**: 
   - **OpenJIJ**: Direct HUBO solving with natural polynomial representation
   - **PyQubo + Neal**: QUBO conversion with one-hot constraints for broad solver compatibility
5. **Solution Validation**: Built-in cost evaluation and constraint checking

### Advantages Over Traditional IP Solvers

- **Polynomial Constraints**: Natural handling of nonlinear constraints like x₀x₁ ≤ 3
- **Quantum-Ready**: Compatible with quantum annealing hardware (D-Wave)
- **Flexible Formulation**: Easy integration of complex objective functions and constraints
- **No Linearization**: Higher-order terms preserved without approximation

### When to Use PyHUBO for Integer Programming

✅ **Good for**: Polynomial objectives, nonlinear constraints, small-to-medium discrete domains, quantum/annealing approaches

⚠️ **Consider alternatives for**: Large-scale linear programs, continuous relaxations, traditional branch-and-bound scenarios

This tutorial showcases how PyHUBO bridges the gap between classical optimization and quantum computing, making it an excellent tool for exploring integer programming problems with quantum and quantum-inspired algorithms!